<a href="https://colab.research.google.com/github/Ciril89/instagram-influencer-graph/blob/master/Similar_AccountsV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd

In [0]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

    100% |████████████████████████████████| 993kB 20.5MB/s 
  Running setup.py bdist_wheel for PyDrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [2]:
! pip install gensim

    100% |████████████████████████████████| 23.6MB 1.7MB/s 
    100% |████████████████████████████████| 1.4MB 16.0MB/s 
    100% |████████████████████████████████| 133kB 13.3MB/s 
    100% |████████████████████████████████| 61kB 25.9MB/s 
    100% |████████████████████████████████| 5.0MB 6.7MB/s 
    100% |████████████████████████████████| 552kB 24.6MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [96]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

True

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [125]:
from google.colab import files
uploaded = files.upload()

Saving accounts_df.csv to accounts_df (1).csv


In [0]:
import io
import pandas as pd
accounts = pd.read_csv(io.StringIO(uploaded['accounts_df.csv'].decode('utf-8')), index_col=['id'])

In [0]:
data = accounts.copy()

In [128]:
data.head(5)

,Unnamed: 0,username,num_posts,bio,num_following,num_followers,is_private,is_verified,profile_pic_url,name,posts_extracted,last_update
id,,,,,,,,,,,,
3044715076,0,ejerciciosdegluteos,421,Epic workouts with the best fitness models of ...,14,133981,False,False,https://scontent-ort2-1.cdninstagram.com/vp/92...,Ejercicios De Gluteos,True,2018-10-18 23:12:44.913404
1639152756,1,sabrinasjourney_,5400,Your favorite coach|Business Mentor\r\n25 Poun...,5324,34049,False,False,https://scontent-ort2-1.cdninstagram.com/vp/98...,Sabrina 🇵🇷🇹🇹,True,2018-10-28 01:23:07.570594
200455459,2,igedz_executioner,611,💪Indonesian heavyweight bodybuilder champion 🥇...,798,53771,False,False,https://scontent-ort2-1.cdninstagram.com/vp/12...,Igede Dharma Susila,True,2018-10-19 11:58:13.840239
5724269803,3,margaux_ragnar,253,🌏📍🇫🇷 France - Sud\r\n👪💍⚒ Viking family\r\n🥗🤦🏽‍...,251,35538,False,False,https://scontent-ort2-1.cdninstagram.com/vp/1a...,margaux_ragnar,True,2018-10-25 12:44:00.267029
319416872,4,smile_xxvii,1125,SMILE XXVII Studios creates content for celebr...,210,33532,False,False,https://scontent-ort2-1.cdninstagram.com/vp/1f...,"Nils Erik, MFA 🇳🇴",True,2018-11-07 06:49:31.839515


In [0]:
data['text'] = accounts['bio']

In [131]:
data.head(5)

,Unnamed: 0,username,num_posts,bio,num_following,num_followers,is_private,is_verified,profile_pic_url,name,posts_extracted,last_update,text
id,,,,,,,,,,,,,
3044715076,0,ejerciciosdegluteos,421,Epic workouts with the best fitness models of ...,14,133981,False,False,https://scontent-ort2-1.cdninstagram.com/vp/92...,Ejercicios De Gluteos,True,2018-10-18 23:12:44.913404,Epic workouts with the best fitness models of ...
1639152756,1,sabrinasjourney_,5400,Your favorite coach|Business Mentor\r\n25 Poun...,5324,34049,False,False,https://scontent-ort2-1.cdninstagram.com/vp/98...,Sabrina 🇵🇷🇹🇹,True,2018-10-28 01:23:07.570594,Your favorite coach|Business Mentor\r\n25 Poun...
200455459,2,igedz_executioner,611,💪Indonesian heavyweight bodybuilder champion 🥇...,798,53771,False,False,https://scontent-ort2-1.cdninstagram.com/vp/12...,Igede Dharma Susila,True,2018-10-19 11:58:13.840239,💪Indonesian heavyweight bodybuilder champion 🥇...
5724269803,3,margaux_ragnar,253,🌏📍🇫🇷 France - Sud\r\n👪💍⚒ Viking family\r\n🥗🤦🏽‍...,251,35538,False,False,https://scontent-ort2-1.cdninstagram.com/vp/1a...,margaux_ragnar,True,2018-10-25 12:44:00.267029,🌏📍🇫🇷 France - Sud\r\n👪💍⚒ Viking family\r\n🥗🤦🏽‍...
319416872,4,smile_xxvii,1125,SMILE XXVII Studios creates content for celebr...,210,33532,False,False,https://scontent-ort2-1.cdninstagram.com/vp/1f...,"Nils Erik, MFA 🇳🇴",True,2018-11-07 06:49:31.839515,SMILE XXVII Studios creates content for celebr...


In [0]:
accounts['bio'] = accounts['bio'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

In [0]:
accounts['text'] = accounts['bio'].str.replace("\r\n", " ")

In [0]:
drop = ['Unnamed: 0',	'username',	'num_posts', 	'bio',	'num_following',	'num_followers',	'is_private',	'is_verified',	'profile_pic_url',	'name',	'posts_extracted',	'last_update'	]

In [0]:
data.drop(drop, axis =1, inplace=True)

In [0]:
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import nltk
import string
import numpy as np
from nltk.corpus import stopwords

cache_english_stopwords=stopwords.words('english')

# df is a pandas dataframe with a column['text'].

def scrubber(df):
    temp_df = df.copy()
    # Remove hyperlinks
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('https?:\/\/.*\/\w*', '', regex=True)
    # Remove hashtags
    # temp_df.loc[:,"text"]=temp_df.loc[:,"text"].replace('#\w*', '', regex=True)
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('#', ' ', regex=True)
    # Remove citations
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('\@\w*', '', regex=True)
    # Remove tickers
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('\$\w*', '', regex=True)
    # Remove punctuation
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('[' + string.punctuation + ']+', '', regex=True)
    # Remove quotes
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('\&*[amp]*\;|gt+', '', regex=True)
    # Remove RT
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('\s+rt\s+', '', regex=True)
    # Remove linebreak, tab, return
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('[\n\t\r]+', ' ', regex=True)
    # Remove via with blank
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('via+\s', '', regex=True)
    # Remove multiple whitespace
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].replace('\s+\s+', ' ', regex=True)
    # Remove empty rows
    temp_df.loc[(temp_df["text"] == '') | (temp_df['text'] == ' ')] = np.nan
    #Remove Emoticons
    temp_df.loc[:, "text"] = temp_df.loc[:, "text"].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
    temp_df = temp_df.dropna()
    return temp_df

In [0]:
clean_text = scrubber(data)


In [0]:
input = clean_text['text']

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [0]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(input)]

In [0]:
tagged_data

In [164]:
max_epochs = 100
vec_size = 50
alpha = 0.025
#dm = Distributed Memory version of Paragraph Vector (PV-DM)
model = Doc2Vec(size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm =1)  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [165]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("fashion model fitness trainer".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)


V1_infer [ 0.00927705  0.05612795 -0.03488512 -0.00738105  0.00738959  0.00785484
 -0.01687912 -0.01879356  0.00421187  0.01816909  0.03531862  0.0299735
  0.03485624  0.02526506 -0.01981493  0.03005032  0.05091463 -0.07181307
 -0.02663237 -0.01983854  0.00097624  0.01170029 -0.00110802  0.008795
  0.05726796 -0.01976479 -0.06185301 -0.00089896 -0.0540213   0.03356068
  0.00038033 -0.05646054 -0.01279378 -0.00936368 -0.03147402 -0.02229959
  0.00528346 -0.0384805  -0.0518216   0.04562261 -0.01469136 -0.07897244
 -0.06720868  0.0386283  -0.03162161  0.05326229  0.00925055 -0.03238136
  0.00458673  0.03128448]


In [170]:
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar(2)
print(similar_doc)




[('3858', 0.7518623471260071), ('475', 0.7455729246139526), ('2756', 0.7323251366615295), ('3083', 0.7236141562461853), ('3773', 0.7048050761222839), ('3546', 0.6992402076721191), ('4327', 0.6969043612480164), ('2817', 0.6956058740615845), ('1819', 0.6948752999305725), ('2639', 0.6881739497184753)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [148]:
# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])

[ 2.4795823e+00  4.8792744e+00 -1.6633567e+00  2.8008206e+00
 -5.6367168e+00 -1.8615071e+00  4.6349570e-01 -1.5153526e+00
  1.5341311e+00  1.3328561e+00  1.8994985e+00  1.0290266e+00
  4.9031644e+00 -2.4187168e-01 -1.8598511e+00  1.2614959e-01
  3.8988919e+00  1.4670449e-04 -1.4193164e+00  3.8588110e-01]


In [171]:
tagged_data[3858]

TaggedDocument(words=['24', 'welsh', 'gb', 'athlete', 'professional', 'bmx', 'rider', 'sponsors', 'gt', 'bikes', 'cannauk', 'fuse', 'protection', 'enquiries', 'email', 'gappyjonescouk'], tags=['3858'])